In [1]:
%conda install langchain -c conda-forge

Channels:
 - conda-forge
 - defaults
Platform: linux-64
Solving environment: done

## Package Plan ##

  environment location: /home/g-999/anaconda3/envs/LangChain

  added / updated specs:
    - langchain


The following packages will be SUPERSEDED by a higher-priority channel:

  certifi            pkgs/main/linux-64::certifi-2024.2.2-~ --> conda-forge/noarch::certifi-2024.2.2-pyhd8ed1ab_0 




Preparing transaction: done
Verifying transaction: done
Executing transaction: done

Note: you may need to restart the kernel to use updated packages.


# Construyendo con LangChain
LangChain permite desarrollar aplicaciones que conectan fuentes externas de datos y cálculos a LLMs (modelos de lenguaje grandes). En este inicio rápido, recorreremos algunas formas diferentes de hacerlo. Comenzaremos con una cadena LLM simple, que se basa únicamente en la información en la plantilla de indicaciones para responder. A continuación, construiremos una cadena de recuperación, que obtiene datos de una base de datos independiente y los pasa a la plantilla de indicaciones. Luego, agregaremos un historial de chat para crear una cadena de recuperación de conversación. Esto permite interactuar de manera similar a un chat con este LLM, para que recuerde preguntas anteriores. Finalmente, construiremos un agente, que utiliza un LLM para determinar si necesita o no obtener datos para responder preguntas. Abordaremos estos temas a un nivel alto, ¡pero hay muchos detalles en todo esto! Enlazaremos a documentos relevantes.

## Cadena LLM
Para esta guía de inicio, proporcionaremos dos opciones: utilizar OpenAI (un modelo popular disponible a través de API) 

Primero, necesitaremos importar el paquete de integración LangChain x OpenAI.

In [2]:
%pip install langchain-openai

Note: you may need to restart the kernel to use updated packages.


In [3]:
# Set env var OPENAI_API_KEY or load from a .env file:
import dotenv

dotenv.load_dotenv()

True

In [4]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI()

Una vez que hayas instalado e inicializado el LLM de tu elección, ¡podemos intentar usarlo! Preguntémosle qué es LangSmith: esto es algo que no estaba presente en los datos de entrenamiento, así que no debería tener una respuesta muy buena.

In [5]:
llm.invoke("¿Cómo puede ayudar Langsmith con las pruebas?")

AIMessage(content='Langsmith puede ayudar con las pruebas de varias maneras:\n\n1. Automatización de pruebas: Langsmith puede automatizar el proceso de ejecución de pruebas, lo que ahorra tiempo y recursos. Puede escribir scripts de prueba y ejecutarlos de manera eficiente para verificar el funcionamiento correcto del software.\n\n2. Generación de datos de prueba: Langsmith puede generar automáticamente datos de prueba realistas y variados para probar diferentes escenarios y casos de uso. Esto ayuda a aumentar la cobertura de prueba y a identificar posibles problemas o errores.\n\n3. Detección de errores: Langsmith utiliza técnicas de aprendizaje automático para analizar los resultados de las pruebas y detectar posibles errores o problemas. Puede identificar patrones o anomalías en los datos de prueba y generar informes detallados sobre los problemas encontrados.\n\n4. Optimización de pruebas: Langsmith puede analizar las pruebas existentes y sugerir mejoras o cambios para optimizar el

También podemos guiar su respuesta con una plantilla de indicaciones. Las plantillas de indicaciones se utilizan para convertir la entrada de usuario sin procesar en una entrada mejor para el LLM.

In [6]:
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_messages([
    ("system", "Eres un escritor de documentación técnica de clase mundial."),
    ("user", "{entrada}")
])

Ahora podemos combinar estas en una cadena LLM simple:

In [7]:
chain = prompt | llm 

Ahora podemos invocarlo y hacer la misma pregunta. Aún no conocerá la respuesta, pero debería responder en un tono más adecuado para un escritor técnico.

In [8]:
chain.invoke({"entrada": "¿Cómo puede ayudar Langsmith con las pruebas?"})

AIMessage(content='Langsmith puede ayudar en las pruebas técnicas de diferentes maneras:\n\n1. Documentación de pruebas: Langsmith puede crear documentación detallada sobre los casos de prueba, los escenarios de prueba y los resultados esperados. Esto ayuda a los equipos de prueba a comprender los requisitos y ejecutar las pruebas de manera efectiva.\n\n2. Automatización de pruebas: Langsmith puede escribir scripts de automatización de pruebas utilizando herramientas como Selenium, Appium, JUnit, TestNG, etc. Estos scripts pueden ejecutar pruebas repetitivas de manera rápida y precisa, ahorrando tiempo y esfuerzo.\n\n3. Documentación de pruebas de rendimiento: Langsmith puede escribir documentación técnica que describa las pruebas de rendimiento y las métricas a medir. Esto ayuda a los equipos a evaluar el rendimiento de una aplicación o sistema y optimizarlo si es necesario.\n\n4. Análisis de resultados de pruebas: Langsmith puede analizar los resultados de las pruebas y proporcionar 

La salida de un modelo de chat (y, por lo tanto, de esta cadena) es un mensaje. Sin embargo, a menudo es mucho más conveniente trabajar con cadenas. Añadamos un simple analizador de salida para convertir el mensaje de chat en una cadena.

In [9]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

Ahora podemos agregar esto a la cadena anterior:

In [10]:
chain = prompt | llm | output_parser

Ahora podemos invocarlo y hacer la misma pregunta. La respuesta ahora será una cadena (en lugar de un ChatMessage).

In [11]:
chain.invoke({"entrada": "¿Cómo puede ayudar Langsmith con las pruebas?"})

'Como escritor de documentación técnica, Langsmith puede ayudar en las pruebas de varias formas:\n\n1. Creación de casos de prueba: Langsmith puede colaborar con los equipos de desarrollo y pruebas para entender los requerimientos y funcionalidades del software. Con esta información, puede crear casos de prueba claros y detallados que ayuden a los equipos de pruebas a validar el correcto funcionamiento del software.\n\n2. Documentación de los resultados de las pruebas: Una vez que los equipos de pruebas ejecuten los casos de prueba, Langsmith puede documentar los resultados de manera clara y concisa. Esto incluye registrar los errores encontrados, su gravedad y cómo reproducirlos, así como también documentar los casos de prueba exitosos.\n\n3. Creación de guías de pruebas: Langsmith puede escribir guías de pruebas detalladas que orienten a los equipos de pruebas sobre cómo ejecutar los casos de prueba correctamente. Estas guías pueden incluir información sobre los pasos a seguir, los d

## Profundizando​
Ahora hemos configurado con éxito una cadena LLM básica. Solo hemos explorado los conceptos básicos de indicaciones, modelos y analizadores de salida; para obtener una comprensión más profunda de todo lo mencionado aquí, consulta esta sección de la documentación.

## Cadena de Recuperación
Para responder adecuadamente a la pregunta original ("¿cómo puede ayudar LangSmith con las pruebas?"), necesitamos proporcionar contexto adicional al LLM. Podemos hacer esto mediante la recuperación. La recuperación es útil cuando tienes demasiados datos para pasar al LLM directamente. Luego puedes usar un recuperador para buscar solo las piezas más relevantes y pasarlas.

En este proceso, buscaremos documentos relevantes desde un Recuperador y luego los pasaremos a la indicación. Un Recuperador puede basarse en cualquier cosa, como una tabla SQL, internet, etc., pero en este caso poblaremos una tienda de vectores y la utilizaremos como un recuperador. Para obtener más información sobre las tiendas de vectores, consulta esta documentación.

Primero, necesitamos cargar los datos que queremos indexar. Para hacer esto, utilizaremos el WebBaseLoader. Esto requiere instalar BeautifulSoup:

In [12]:
%conda install beautifulsoup4

Channels:
 - defaults
 - conda-forge
Platform: linux-64
Solving environment: done

## Package Plan ##

  environment location: /home/g-999/anaconda3/envs/LangChain

  added / updated specs:
    - beautifulsoup4


The following packages will be SUPERSEDED by a higher-priority channel:

  certifi            conda-forge/noarch::certifi-2024.2.2-~ --> pkgs/main/linux-64::certifi-2024.2.2-py311h06a4308_0 




Preparing transaction: done
Verifying transaction: done
Executing transaction: done

Note: you may need to restart the kernel to use updated packages.


Después de eso, podemos importar y usar WebBaseLoader.

In [13]:
from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader("https://docs.smith.langchain.com/overview")

docs = loader.load()

A continuación, necesitamos indexarlo en una tienda de vectores. Esto requiere algunos componentes, en particular, un modelo de incrustación y una tienda de vectores.

En cuanto a los modelos de incrustación, proporcionamos nuevamente ejemplos de acceso a través de OpenAI o mediante modelos locales.


Asegúrate de tener instalado el paquete `langchain_openai` y las variables de entorno adecuadas configuradas (son las mismas que se necesitan para el LLM).

In [14]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings()

Ahora podemos utilizar este modelo de incrustación para ingresar documentos en una tienda de vectores. Utilizaremos una tienda de vectores local y sencilla, FAISS, por simplicidad.

Primero, necesitamos instalar los paquetes necesarios para ello:

In [15]:
%conda install faiss-cpu

Channels:
 - defaults
Platform: linux-64
Solving environment: done

# All requested packages already installed.


Note: you may need to restart the kernel to use updated packages.


Luego podemos construir nuestro índice:

In [16]:
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter


text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(docs)
vector = FAISS.from_documents(documents, embeddings)

Ahora que tenemos estos datos indexados en una tienda de vectores, crearemos una cadena de recuperación. Esta cadena tomará una pregunta entrante, buscará documentos relevantes y luego pasará esos documentos junto con la pregunta original a un LLM y le pedirá que responda la pregunta original.

Primero, configuremos la cadena que toma una pregunta y los documentos recuperados y genera una respuesta.

In [17]:
from langchain.chains.combine_documents import create_stuff_documents_chain

prompt = ChatPromptTemplate.from_template("""Responda la siguiente pregunta basándose únicamente en el contexto proporcionado:

<context>
{context}
</context>

Question: {input}""")

document_chain = create_stuff_documents_chain(llm, prompt)

In [18]:
from langchain_core.documents import Document

document_chain.invoke({
    "input": "¿Cómo puede ayudar Langsmith con las pruebas?",
    "context": [Document(page_content="Langsmith puede permitirle visualizar los resultados de las pruebas")]
})

'Langsmith puede permitirle visualizar los resultados de las pruebas.'

Sin embargo, queremos que los documentos provengan primero del recuperador que acabamos de configurar. De esta manera, para una pregunta dada, podemos utilizar el recuperador para seleccionar dinámicamente los documentos más relevantes y pasarlos.

In [19]:
from langchain.chains import create_retrieval_chain

retriever = vector.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)

Ahora podemos invocar esta cadena. Esto devuelve un diccionario: la respuesta del LLM se encuentra en la clave "answer".

In [20]:
response = retrieval_chain.invoke({"input":"¿Cómo puede ayudar Langsmith con las pruebas?"})
print(response["answer"])

# LangSmith offers several features that can help with testing:...

LangSmith puede ayudar con las pruebas de varias formas. 

En primer lugar, LangSmith proporciona una visualización clara de las entradas y salidas exactas de todas las llamadas al modelo de lenguaje (LLM). Esto permite comprender fácilmente cómo se están procesando los datos y facilita la depuración de posibles errores en la lógica de formato o transformaciones inesperadas de las entradas del usuario.

Además, LangSmith ofrece una función de "Playground" que permite editar la entrada de una llamada al LLM y ver cómo afecta la salida. Esto es útil para probar diferentes cambios y observar los resultados sin tener que copiar y pegar la entrada en un entorno separado.

LangSmith también simplifica la carga de conjuntos de datos para probar cambios en las cadenas de comandos o en las respuestas del modelo. Permite ejecutar la cadena de comandos en los puntos de datos del conjunto de datos y visualizar los resultados. También proporciona herramientas para asignar comentarios y marcar las e

## Cadena de Recuperación de Conversación
La cadena que hemos creado hasta ahora solo puede responder preguntas individuales. Uno de los principales tipos de aplicaciones de LLM que las personas están desarrollando son los chatbots. Entonces, ¿cómo convertimos esta cadena en una que pueda responder preguntas de seguimiento?

Todavía podemos utilizar la función create_retrieval_chain, pero necesitamos cambiar dos cosas:

1. El método de recuperación ahora no solo debe funcionar en la entrada más reciente, sino que debe tener en cuenta toda la historia.
2. La cadena final del LLM también debe tener en cuenta toda la historia.

Actualización de la Recuperación

Para actualizar la recuperación, crearemos una nueva cadena. Esta cadena tomará la entrada más reciente (input) y el historial de la conversación (chat_history) y utilizará un LLM para generar una consulta de búsqueda.

In [21]:
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder

# First we need a prompt that we can pass into an LLM to generate this search query

prompt = ChatPromptTemplate.from_messages([
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
    ("user", "Dada la conversación anterior, genere una consulta de búsqueda para buscar información relevante para la conversación.")
])
retriever_chain = create_history_aware_retriever(llm, retriever, prompt)

Podemos probar esto pasando una instancia en la que el usuario hace una pregunta de seguimiento.

In [22]:
from langchain_core.messages import HumanMessage, AIMessage

chat_history = [HumanMessage(content="¿Puede LangSmith ayudarme a probar mis solicitudes de LLM?"), AIMessage(content="¡Si!")]
retriever_chain.invoke({
    "chat_history": chat_history,
    "input": "Dime como"
})

[Document(page_content="Skip to main content🦜️🛠️ LangSmith DocsPython DocsJS/TS DocsSearchGo to AppLangSmithOverviewTracingTesting & EvaluationOrganizationsHubLangSmith CookbookRelease NotesOverviewOn this pageLangSmith Overview and User GuideBuilding reliable LLM applications can be challenging. LangChain simplifies the initial setup, but there is still work needed to bring the performance of prompts, chains and agents up the level where they are reliable enough to be used in production.Over the past two months, we at LangChain have been building and using LangSmith with the goal of bridging this gap. This is our tactical user guide to outline effective ways to use LangSmith and maximize its benefits.On by default\u200bAt LangChain, all of us have LangSmith’s tracing running in the background by default. On the Python side, this is achieved by setting environment variables, which we establish whenever we launch a virtual environment or open our bash shell and leave them set. The same 

Deberías ver que esto devuelve documentos sobre pruebas en LangSmith. Esto se debe a que el LLM generó una nueva consulta, combinando el historial del chat con la pregunta de seguimiento.

Ahora que tenemos este nuevo recuperador, podemos crear una nueva cadena para continuar la conversación teniendo en cuenta estos documentos recuperados.

In [23]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "Responda las preguntas del usuario según el siguiente contexto:\n\n{context}"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
])
document_chain = create_stuff_documents_chain(llm, prompt)

retrieval_chain = create_retrieval_chain(retriever_chain, document_chain)

Ahora a probar

In [24]:
chat_history = [HumanMessage(content="¿Puede LangSmith ayudarme a probar mis solicitudes de LLM?"), AIMessage(content="¡SI!")]
retrieval_chain.invoke({
    "chat_history": chat_history,
    "input": "Dime como"
})

{'chat_history': [HumanMessage(content='¿Puede LangSmith ayudarme a probar mis solicitudes de LLM?'),
  AIMessage(content='¡SI!')],
 'input': 'Dime como',
 'context': [Document(page_content="Skip to main content🦜️🛠️ LangSmith DocsPython DocsJS/TS DocsSearchGo to AppLangSmithOverviewTracingTesting & EvaluationOrganizationsHubLangSmith CookbookRelease NotesOverviewOn this pageLangSmith Overview and User GuideBuilding reliable LLM applications can be challenging. LangChain simplifies the initial setup, but there is still work needed to bring the performance of prompts, chains and agents up the level where they are reliable enough to be used in production.Over the past two months, we at LangChain have been building and using LangSmith with the goal of bridging this gap. This is our tactical user guide to outline effective ways to use LangSmith and maximize its benefits.On by default\u200bAt LangChain, all of us have LangSmith’s tracing running in the background by default. On the Python si

Podemos ver que esto da una respuesta coherente: ¡hemos convertido con éxito nuestra cadena de recuperación en un chatbot!

## Agente
Hasta ahora, hemos creado ejemplos de cadenas, donde cada paso se conoce de antemano. Lo último que crearemos es un agente, donde el LLM decide qué pasos tomar.

NOTA: para este ejemplo, solo mostraremos cómo crear un agente utilizando modelos de OpenAI, ya que los modelos locales aún no son lo suficientemente confiables.

Una de las primeras cosas que debes hacer al construir un agente es decidir a qué herramientas debería tener acceso. Para este ejemplo, daremos al agente acceso a dos herramientas:

1. El recuperador que acabamos de crear. Esto le permitirá responder fácilmente preguntas sobre LangSmith.
2. Una herramienta de búsqueda. Esto le permitirá responder fácilmente preguntas que requieran información actualizada.

Primero, configuremos una herramienta para el recuperador que acabamos de crear:

In [25]:
from langchain.tools.retriever import create_retriever_tool

retriever_tool = create_retriever_tool(
    retriever,
    "langsmith_search",
    "Buscar información sobre LangSmith. ¡Para cualquier pregunta sobre LangSmith, debes utilizar esta herramienta!",
)

La herramienta de búsqueda que utilizaremos es Tavily. Esto requerirá una clave API (tienen un generoso nivel gratuito). Después de crearlo en su plataforma, debes configurarlo como una variable de entorno:

Actualizamos el .env con la clave de tavily

In [26]:
import dotenv

dotenv.load_dotenv()

True

In [27]:
from langchain_community.tools.tavily_search import TavilySearchResults

search = TavilySearchResults()

Ahora podemos crear una lista de las herramientas con las que queremos trabajar:

In [28]:
tools = [retriever_tool, search]

Ahora que tenemos las herramientas, podemos crear un agente para usarlas. Repasaremos esto rápidamente; para profundizar en lo que está sucediendo exactamente, consulte la documentación de introducción del agente. 

Instale primero el HUB langchain

In [31]:
%pip install langchainhub

Note: you may need to restart the kernel to use updated packages.


Ahora podemos usarlo para obtener un mensaje predefinido.

In [32]:
from langchain_openai import ChatOpenAI
from langchain import hub
from langchain.agents import create_openai_functions_agent
from langchain.agents import AgentExecutor

# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/openai-functions-agent")
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
agent = create_openai_functions_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

¡Ahora podemos invocar al agente y ver cómo responde! Podemos hacerle preguntas sobre LangSmith:

In [33]:
agent_executor.invoke({"input": "¿Cómo puede ayudar Langsmith con las pruebas?"})



> Entering new AgentExecutor chain...
LangSmith puede ayudar con las pruebas de varias maneras:

1. Generación automática de casos de prueba: LangSmith puede generar automáticamente casos de prueba para diferentes escenarios y condiciones. Esto ahorra tiempo y esfuerzo al crear casos de prueba manualmente.

2. Ejecución de pruebas automatizadas: LangSmith puede ejecutar pruebas automatizadas para verificar el funcionamiento correcto de una aplicación o sistema. Esto incluye la ejecución de pruebas de regresión, pruebas de integración y pruebas de rendimiento.

3. Análisis de resultados de pruebas: LangSmith puede analizar los resultados de las pruebas y proporcionar informes detallados sobre los errores encontrados, el rendimiento del sistema y la cobertura de las pruebas.

4. Optimización de pruebas: LangSmith puede ayudar a optimizar las pruebas identificando áreas críticas que requieren una mayor cobertura de pruebas y sugiriendo mejoras en los casos de prueba existentes.

5. Inte

{'input': '¿Cómo puede ayudar Langsmith con las pruebas?',
 'output': 'LangSmith puede ayudar con las pruebas de varias maneras:\n\n1. Generación automática de casos de prueba: LangSmith puede generar automáticamente casos de prueba para diferentes escenarios y condiciones. Esto ahorra tiempo y esfuerzo al crear casos de prueba manualmente.\n\n2. Ejecución de pruebas automatizadas: LangSmith puede ejecutar pruebas automatizadas para verificar el funcionamiento correcto de una aplicación o sistema. Esto incluye la ejecución de pruebas de regresión, pruebas de integración y pruebas de rendimiento.\n\n3. Análisis de resultados de pruebas: LangSmith puede analizar los resultados de las pruebas y proporcionar informes detallados sobre los errores encontrados, el rendimiento del sistema y la cobertura de las pruebas.\n\n4. Optimización de pruebas: LangSmith puede ayudar a optimizar las pruebas identificando áreas críticas que requieren una mayor cobertura de pruebas y sugiriendo mejoras en l

Podemos preguntarle sobre el clima:

In [36]:
agent_executor.invoke({"input": "¿Cuál es el clima en NY?"})



> Entering new AgentExecutor chain...

Invoking: `tavily_search_results_json` with `{'query': 'clima en NY'}`


[{'url': 'https://weatherspark.com/h/y/23912/2024/Historical-Weather-during-2024-in-New-York-City-New-York-United-States', 'content': '2024 Weather History in New York City New York, United States  New York City Temperature History 2024 Hourly Temperature in 2024 in New York City  Daily Precipitation in 2024 in New York City Observed Weather in 2024 in New York City  Humidity Comfort Levels in 2024 in New York City Wind Speed in 2024 in New York CityLatest Report — 10:51 AM Fri, Feb 2, 2024 17 min ago UTC 15:51 Call Sign KEWR Temp. 42.1 °F very cold Precipitation - last 1 hr 0.02 in Wind 11.5 mph gentle breeze Wind Dir. 360 deg, N Cloud Cover Overcast 3,000 ft Mostly Cloudy 700 ft Mostly Cloudy 1,500 ft'}]Según los últimos informes, el clima en Nueva York es de 42.1 °F (5.6 °C) con viento de 11.5 mph (18.5 km/h) en dirección norte. La cobertura de nubes es mayormente nublad

{'input': '¿Cuál es el clima en NY?',
 'output': 'Según los últimos informes, el clima en Nueva York es de 42.1 °F (5.6 °C) con viento de 11.5 mph (18.5 km/h) en dirección norte. La cobertura de nubes es mayormente nublado con algunas nubes a 3,000 pies y 700 pies.'}

In [37]:
chat_history = [HumanMessage(content="¿Puede LangSmith ayudarme a probar mis solicitudes de LLM?"), AIMessage(content="¡Si!")]
agent_executor.invoke({
    "chat_history": chat_history,
    "input": "Dime como"
})



> Entering new AgentExecutor chain...
Para probar tus solicitudes de LLM (Lenguaje de Modelado de Lenguajes) con LangSmith, puedes seguir estos pasos:

1. Define tu solicitud de LLM: Primero, debes tener claro qué tipo de solicitud de LLM quieres probar. Puedes especificar el tipo de modelo de lenguaje que deseas utilizar, las entradas y salidas esperadas, y cualquier otra información relevante.

2. Utiliza la herramienta de búsqueda de LangSmith: Puedes utilizar la función `langsmith_search` para buscar información sobre LangSmith y obtener respuestas a tus preguntas específicas. Esta función te proporcionará información detallada sobre cómo utilizar LangSmith para probar tus solicitudes de LLM.

3. Consulta la documentación de LangSmith: LangSmith proporciona documentación detallada sobre cómo utilizar su plataforma para probar solicitudes de LLM. Puedes acceder a esta documentación en su sitio web oficial o a través de la herramienta de búsqueda de LangSmith.

4. Prueba tus solici

{'chat_history': [HumanMessage(content='¿Puede LangSmith ayudarme a probar mis solicitudes de LLM?'),
  AIMessage(content='¡Si!')],
 'input': 'Dime como',
 'output': 'Para probar tus solicitudes de LLM (Lenguaje de Modelado de Lenguajes) con LangSmith, puedes seguir estos pasos:\n\n1. Define tu solicitud de LLM: Primero, debes tener claro qué tipo de solicitud de LLM quieres probar. Puedes especificar el tipo de modelo de lenguaje que deseas utilizar, las entradas y salidas esperadas, y cualquier otra información relevante.\n\n2. Utiliza la herramienta de búsqueda de LangSmith: Puedes utilizar la función `langsmith_search` para buscar información sobre LangSmith y obtener respuestas a tus preguntas específicas. Esta función te proporcionará información detallada sobre cómo utilizar LangSmith para probar tus solicitudes de LLM.\n\n3. Consulta la documentación de LangSmith: LangSmith proporciona documentación detallada sobre cómo utilizar su plataforma para probar solicitudes de LLM. Pue

## Sirviendo con LangServe
Ahora que hemos construido una aplicación, necesitamos ponerla en funcionamiento. Ahí es donde entra LangServe. LangServe ayuda a los desarrolladores a implementar cadenas de LangChain como una API REST. No es necesario utilizar LangServe para usar LangChain, pero en esta guía mostraremos cómo puedes implementar tu aplicación con LangServe.

Mientras que la primera parte de esta guía estaba destinada a ejecutarse en un cuaderno Jupyter, ahora saldremos de eso. Crearemos un archivo Python y luego interactuaremos con él desde la línea de comandos.

In [38]:
%pip install "langserve[all]"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.1/92.1 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.7/138.7 kB 3.5 MB/s eta 0:00:00 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 512.8/512.8 kB 3.9 MB/s eta 0:00:003.8 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 6.2 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


## Servidor 
Para crear un servidor para nuestra aplicación, crearemos un archivo [serve.py](http://localhost:8889/lab/tree/Documentos/GitClone/CookBook/LangChain/serve.py). 
Esto contendrá nuestra lógica para servir nuestra aplicación. 
Consta de tres cosas: 
1. La definición de nuestra cadena que acabamos de crear arriba
2. Nuestra aplicación FastAPI
3. Una definición de una ruta desde la cual servir a la cadena, que se hace con langserve.add_routes

deberíamos ver nuestra [cadena](http://localhost:8889/lab/tree/Documentos/GitClone/CookBook/LangChain/serve.py) siendo atendida en localhost:8000.

## Playground 
Cada servicio LangServe viene con una interfaz de usuario integrada simple para configurar e invocar la aplicación con salida de transmisión y visibilidad de los pasos intermedios. 
Dirígete a http://localhost:8000/agent/playground/ ¡para probarlo! Pase la misma pregunta que antes: "¿Cómo puede ayudar Langsmith con las pruebas?" - y debería responder igual que antes.

## Cliente 
Ahora configuremos un cliente para interactuar mediante programación con nuestro servicio. Podemos hacer esto fácilmente con [langserve.RemoteRunnable](/docs/langserve#client) Al usar esto, podemos interactuar con la cadena servida como si se estuviera ejecutando en el lado del cliente.

In [41]:
from langserve import RemoteRunnable

remote_chain = RemoteRunnable("http://localhost:8000/agent/")
remote_chain.invoke({
    "input": "¿Cómo puede ayudar Langsmith con las pruebas?",
    "chat_history": []  # Providing an empty list as this is the first call
})

{'output': 'LangSmith puede ayudar con las pruebas de varias maneras:\n\n1. Generación automática de casos de prueba: LangSmith puede generar automáticamente casos de prueba basados en las especificaciones y requisitos del software. Esto ahorra tiempo y esfuerzo en la creación manual de casos de prueba.\n\n2. Ejecución de pruebas automatizadas: LangSmith puede ejecutar pruebas automatizadas para verificar el funcionamiento correcto del software. Esto incluye pruebas de regresión, pruebas de integración y pruebas de unidad.\n\n3. Análisis de cobertura de código: LangSmith puede analizar la cobertura de código para identificar áreas no probadas del software. Esto ayuda a garantizar una cobertura completa de las pruebas y a identificar posibles problemas o errores.\n\n4. Generación de informes de pruebas: LangSmith puede generar informes detallados de las pruebas realizadas, incluyendo resultados, errores encontrados y métricas de calidad. Esto facilita el seguimiento y la gestión de las 